In [1]:
from google.colab import drive
from os.path import join  

def mount_drive(ROOT):
    drive.mount(ROOT, force_remount=True)

ROOT = '/content/drive'
mount_drive(ROOT)

MY_GOOGLE_DRIVE_PATH = 'MyDrive/RL_NLP/' 
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)
print("PROJECT_PATH: ", PROJECT_PATH)   
%cd "{PROJECT_PATH}"

Mounted at /content/drive
PROJECT_PATH:  /content/drive/MyDrive/RL_NLP/
/content/drive/MyDrive/RL_NLP


In [2]:
!pip3 install virtualenv
!virtualenv -p python3.8.10 python38

!source /content/drive/MyDrive/Colab/RL_NLP/python38/bin/activate
!python --version

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
created virtual environment CPython3.8.10.final.0-64 in 16411ms
  creator CPython3Posix(dest=/content/drive/MyDrive/RL_NLP/python38, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.0, setuptools==67.1.0, wheel==0.38.4
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
/bin/bash: /content/drive/MyDrive/Colab/RL_NLP/python38/bin/activate: No such file or directory
Python 3.8.10


In [3]:
!pip install -r "{PROJECT_PATH}/metadata/requirements.txt"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [5]:
!git clone https://github.com/Farama-Foundation/gym-examples
%cd gym-examples
!pip install -e .
%cd ..

fatal: destination path 'gym-examples' already exists and is not an empty directory.
/content/drive/MyDrive/RL_NLP/gym-examples
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/RL_NLP/gym-examples
  Preparing metadata (setup.py) ... done
  Attempting uninstall: gym-examples
    Found existing installation: gym-examples 0.0.1
    Can't uninstall 'gym-examples'. No files were found to uninstall.
  Running setup.py develop for gym-examples
/content/drive/MyDrive/RL_NLP


In [ ]:
import gym
import gym_examples
env = gym.make('gym_examples/RlNlpWorld-v0')